# 06 - Cohere Interview Response Evaluation
`Author: Abdlazeez Jimoh`

In [3]:
import json
from pprint import pprint
from typing import Literal, TypedDict

import cohere

In [8]:
COHERE_API_KEY = "YOUR_API_KEY"

In [9]:
class Question(TypedDict):
    question: str
    type: Literal["personal", "role-specific", "behavioral", "situational"]


class Evaluation(TypedDict):
    evaluation: Literal["good", "average", "bad"]
    feedback: str | None
    reason: str | None
    samples: list[str] | None

In [10]:
class CohereResponseEvaluationAgent:
    def __init__(self):
        self.client = cohere.Client(COHERE_API_KEY)
        self.system_prompt = """You are an interviewer evaluating a candidate's response to an interview question. Your task is to:
- Evaluate the candidate's response on the scale of "good", "average", and "bad".
- Provide a reason for why it's categorized as good, average, or bad.
- Offer constructive feedback or suggestions for improvement.
- Provide 2 samples of good responses.

You will be provided with an interview question and a candidate response.

Evaluate and provide output in the following JSON format:
{{
    "evaluation": "good, average, or bad",
    "reason": "Reason why it's good, average, or bad",
    "feedback": "Feedback or suggestions for improvement",
    "samples": [
        "Good response 1", 
        "Good response 2"
    ]
}}


===
QUESTION:
{question}

RESPONSE: 
{response}"""

    def __call__(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def run(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        # Generate questions
        evaluation = self._generate(question, response)

        return evaluation

    def _generate(self, question: str, response: str) -> Evaluation | None:
        """
        Evaluate a candidate's response to an interview question.

        Args:
            question (str): The interview question.
            response (str): The candidate's response.

        Returns:
            Evaluation | None: The evaluation of the candidate's response or None if an error occurred.
        """

        try:
            output = self.client.generate(
                model="command",
                prompt=self.system_prompt.format(question=question, response=response),
                temperature=1,
                max_tokens=1024,
            )

            evaluations = json.loads(output.generations[0].text)

            return evaluations
        except Exception as e:
            return None

In [11]:
response_evaluator = CohereResponseEvaluationAgent()
evaluation = response_evaluator.run(
    "What motivated you to pursue a career in software engineering at a startup in San Francisco?",
    "I've always been interested in technology and startups. I've been working in the tech industry for the past 5 years and I've always wanted to work at a startup. I'm really excited about the opportunity to work at a startup in San Francisco."
)

pprint(evaluation)

{'evaluation': 'good',
 'feedback': 'It would be helpful to hear more about what specifically '
             'motivated them about working at a startup over a larger company, '
             'or more specifically about working in San Francisco. This would '
             'provide more insight into their values and goals.',
 'reason': 'The candidate provides a clear reason for their career choice, '
           'stating their interest in both technology and startups. This '
           'demonstrates a sincere passion and alignment with the field of '
           'software engineering and the startup environment.',
 'samples': ['I pursued a career in software engineering at a startup in San '
             'Francisco because I believe that startups provide an environment '
             'of rapid personal and professional growth, where I can take on '
             'diverse responsibilities and have a direct impact on the '
             "company's trajectory. I am excited about the opportunities

In [12]:
evaluation = response_evaluator.run(
    "How do you prioritize tasks when leading a team of data scientists and engineers on multiple projects?",
    "I prioritize tasks by importance and urgency. I also make sure to communicate with my team about what needs to be done and when it needs to be done. I also make sure to communicate with my team about what needs to be done and when it needs to be done.",
)

pprint(evaluation)

{'evaluation': 'good',
 'feedback': 'It would be helpful to hear more about the specific tools or '
             'frameworks the candidate utilizes to manage tasks and team '
             'communication. Additionally, discussing how they handle '
             'situations where priorities or deadlines shift could provide '
             'insight into their adaptability and problem-solving skills.',
 'reason': 'The candidate discusses how they prioritize tasks by importance '
           'and urgency, which is a good approach to managing tasks. They also '
           'highlight the importance of communication with their team, which '
           'is crucial for effective task management and collaboration.',
 'samples': ['I utilize the Eisenhower Matrix (urgent vs. important) to '
             'efficiently organize and delegate tasks. This ensures that both '
             'urgent and important tasks are given proper attention, while '
             'also providing a framework for my team to f